# Data Analyst Jobs Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree

from xgboost import XGBRegressor

In [2]:
def disp_unique_col_val(columns):
    for col in columns:
        uniques = df[col].unique()
        print(col)
        print('=' * 30)
        for unique in uniques:
            print(unique)
        if (col != df.columns[-1]): print('\n')

## Model Import

In [3]:
df = pd.read_csv('assets/data_analyst_jobs.csv')

## Model Inspection

In [4]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         2253 non-null   int64  
 1   Job Title          2253 non-null   object 
 2   Salary Estimate    2253 non-null   object 
 3   Job Description    2253 non-null   object 
 4   Rating             2253 non-null   float64
 5   Company Name       2252 non-null   object 
 6   Location           2253 non-null   object 
 7   Headquarters       2253 non-null   object 
 8   Size               2253 non-null   object 
 9   Founded            2253 non-null   int64  
 10  Type of ownership  2253 non-null   object 
 11  Industry           2253 non-null   object 
 12  Sector             2253 non-null   object 
 13  Revenue            2253 non-null   object 
 14  Competitors        2253 non-null   object 
 15  Easy Apply         2253 non-null   object 
dtypes: float64(1), int64(2),

## Model Cleaning

### Drop Unused Columns

In [6]:
col_drop_list = [df.columns[0], df.columns[3], df.columns[5], df.columns[14], df.columns[15]]
df = df.drop(columns=col_drop_list)
print(col_drop_list)
disp_unique_col_val(df.columns[1:])

['Unnamed: 0', 'Job Description', 'Company Name', 'Competitors', 'Easy Apply']
Salary Estimate
$37K-$66K (Glassdoor est.)
$46K-$87K (Glassdoor est.)
$51K-$88K (Glassdoor est.)
$51K-$87K (Glassdoor est.)
$59K-$85K (Glassdoor est.)
$43K-$76K (Glassdoor est.)
$60K-$110K (Glassdoor est.)
$41K-$78K (Glassdoor est.)
$45K-$88K (Glassdoor est.)
$73K-$127K (Glassdoor est.)
$84K-$90K (Glassdoor est.)
$27K-$52K (Glassdoor est.)
$42K-$74K (Glassdoor est.)
$77K-$132K (Glassdoor est.)
$98K-$114K (Glassdoor est.)
$48K-$96K (Glassdoor est.)
$26K-$47K (Glassdoor est.)
$31K-$59K (Glassdoor est.)
$47K-$81K (Glassdoor est.)
$43K-$69K (Glassdoor est.)
$49K-$112K (Glassdoor est.)
$30K-$54K (Glassdoor est.)
$55K-$103K (Glassdoor est.)
$37K-$70K (Glassdoor est.)
$57K-$103K (Glassdoor est.)
$35K-$45K (Glassdoor est.)
$42K-$66K (Glassdoor est.)
$65K-$81K (Glassdoor est.)
$113K-$132K (Glassdoor est.)
$42K-$63K (Glassdoor est.)
$60K-$66K (Glassdoor est.)
$73K-$82K (Glassdoor est.)
$67K-$92K (Glassdoor est.)
$42K-

### Replace Values to NaN

In [7]:
df = df.replace(['-1', '-1.0', -1, 'Unknown', 'Unknown / Non-Applicable'], np.nan)
disp_unique_col_val(df.columns)

Job Title
Data Analyst, Center on Immigration and Justice (CIJ)
Quality Data Analyst
Senior Data Analyst, Insights & Analytics Team [Customer Operations]
Data Analyst
Reporting Data Analyst
Business/Data Analyst (FP&A)
Data Science Analyst
Data Analyst, Merchant Health
DATA ANALYST
Senior Data Analyst
Investment Advisory Data Analyst
Sustainability Data Analyst
Clinical Data Analyst
DATA PROGRAMMER/ANALYST
Product Analyst, Data Science
Data Analyst - Intex Developer
Entry Level / Jr. Data Analyst
Data + Business Intelligence Analyst
Data Analyst, Product
Data Analyst Entry Level
Data Science Analyst, Capital Markets
Data Analyst (Games)
Analyst/Associate Global Markets Credit Data Analyst
Data Business Analyst
Data Analyst with Excel/DAX/ PowerBI experience- Fulltime
Data Science Analyst/Engineer
Business Analyst, Data Platforms
Behavioral Data Analyst
Data Analyst -1+ year Contract - NYC
Advertising Data Analyst
Market Data Reporting Analyst
Senior Analyst, Data Science
Senior Data An

Data & Reporting Analyst, Distribution Planning & Operations
Epidemiologist - Data Analyst
Contract Data Analyst
Epidemiology Data Analyst
Data Business Solutions Analyst
Risk Data Analyst
Epidemiology/Data Analyst, San Antonio, TX, US, ID
Senior Data Science Analyst
Data Analyst ( SAS,SQL)
Data Analyst - Top Secret w/ SCI Eligibility
Epidemiology/Data Analyst
Data Security Analyst
Data Analyst/Engineers
Senior Contract Data Analyst
IT Data Science Analyst
Configuration Data Analyst, (Secret Clearance Required) San Antonio, TX, US ID
TECHNICAL DATA ANALYST
Configuration Data Analyst, (Secret Clearance Required) San An with Security Clearance
Senior Risk Analyst-Data
Functional Analyst (CECL Data)
Data Analyst with Security Clearance
Configuration Data Analyst, San Antonio, TX, US ID#17897 with Security Clearance
(REMOTE) Sr. Risk Analyst (IT, IS and/or Data expertise)
Sr. Risk Analyst (IT, IS and/or Data expertise)
Navy Maintenance Data Systems Analyst
Division Data and Financial Analy

Marin City, CA
Concord, CA
San Ramon, CA
Brentwood, CA
Emeryville, CA
East Brunswick, NJ
Chesterfield, MO
Keswick, VA
South Windsor, CT
Minneapolis, MN
Vienna, Austria
Hercules, CA
Faridabad, India
South Melbourne, Australia
South San Francisco, CA
Pleasant Hill, CA
Worthington, OH
Carmel, IN
Wilmington, DE
Indianapolis, IN
Feusisberg, Switzerland
Edmond, OK
Mishawaka, IN
Moosic, PA
Redmond, WA
Issaquah, WA
Burnsville, MN
Kent, WA
Kyoto, Japan
Tigard, OR
Athens, GA
Cooper City, FL
Gilbert, AZ
Louisville, CO
Lakewood, CO
Rochester, MN
Monroe, LA
Lakeland, FL
Aurora, IL
Beaverton, OR
West Des Moines, IA
West Chester, OH
Broomfield, CO


Size
201 to 500 employees
10000+ employees
1001 to 5000 employees
501 to 1000 employees
5001 to 10000 employees
1 to 50 employees
51 to 200 employees
nan


Founded
1961.0
1893.0
2003.0
2002.0
2009.0
2014.0
2001.0
1914.0
1896.0
2013.0
1841.0
nan
1884.0
2004.0
2016.0
1996.0
1958.0
2006.0
1998.0
1969.0
2011.0
2017.0
2005.0
1904.0
2018.0
1956.0
2007.0
1990.0


### Drop NaN

In [8]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 2252
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          1205 non-null   object 
 1   Salary Estimate    1205 non-null   object 
 2   Rating             1205 non-null   float64
 3   Location           1205 non-null   object 
 4   Headquarters       1205 non-null   object 
 5   Size               1205 non-null   object 
 6   Founded            1205 non-null   float64
 7   Type of ownership  1205 non-null   object 
 8   Industry           1205 non-null   object 
 9   Sector             1205 non-null   object 
 10  Revenue            1205 non-null   object 
dtypes: float64(2), object(9)
memory usage: 113.0+ KB


## Preparation

In [9]:
x = pd.DataFrame()
y = pd.DataFrame()

### Y

#### Salary

In [10]:
# Salary

# salaries = list(df[df.columns[1]].unique())
# salaries.sort(key=lambda x: (int(re.findall('\d+', x)[0]), int(re.findall('\d+', x)[1])))
# replacement_index = {}
# for i in range(len(salaries)):
#     replacement_index[salaries[i]] = i
# x['Employee Number'] = df[df.columns[1]].replace(replacement_index)
# print(json.dumps(replacement_index, indent=4))
# x.head()

salaries = df[df.columns[1]].copy()
salary_l = []
salary_u = []
for salary in salaries:
    lim_l, lim_u = re.findall('\d+', salary)
    salary_l.append(int(lim_l))
    salary_u.append(int(lim_u))
y['Salary Lower'] = salary_l
y['Salary Upper'] = salary_u
y

,Salary Lower,Salary Upper
0,37,66
1,37,66
2,37,66
3,37,66
4,37,66
...,...,...
1200,78,104
1201,78,104
1202,78,104
1203,78,104


### X

#### Rating

In [11]:
x['Rating'] = df[df.columns[2]] 
x.head()

,Rating
0,3.2
1,3.8
3,4.1
4,3.9
7,3.7


#### Location

In [12]:
# Location
locations = np.sort(df[df.columns[3]].unique())
replacement_index = {}
for i in range(len(locations)):
    replacement_index[locations[i]] = i
x['Location'] = df[df.columns[3]].replace(replacement_index)
print(json.dumps(replacement_index, indent=4))
x.head()

{
    "Alachua, FL": 0,
    "Alameda, CA": 1,
    "Allen, TX": 2,
    "American Fork, UT": 3,
    "Arcadia, CA": 4,
    "Arlington, TX": 5,
    "Athens, GA": 6,
    "Aurora, CO": 7,
    "Austin, TX": 8,
    "Azusa, CA": 9,
    "Beech Grove, IN": 10,
    "Bellevue, WA": 11,
    "Bensalem, PA": 12,
    "Berkeley Heights, NJ": 13,
    "Berkeley, CA": 14,
    "Berwyn, PA": 15,
    "Beverly Hills, CA": 16,
    "Blue Bell, PA": 17,
    "Boothwyn, PA": 18,
    "Boulder, CO": 19,
    "Brea, CA": 20,
    "Broadview, IL": 21,
    "Bronx, NY": 22,
    "Brooklyn, NY": 23,
    "Broomfield, CO": 24,
    "Burbank, CA": 25,
    "Burlingame, CA": 26,
    "Burlingame, KS": 27,
    "Camden, NJ": 28,
    "Carmel, IN": 29,
    "Carrollton, TX": 30,
    "Centennial, CO": 31,
    "Cerritos, CA": 32,
    "Chandler, AZ": 33,
    "Charlotte, NC": 34,
    "Chesapeake, VA": 35,
    "Chicago, IL": 36,
    "Columbus, OH": 37,
    "Conshohocken, PA": 38,
    "Coppell, TX": 39,
    "Culver City, CA": 40,
    "Cuperti

,Rating,Location
0,3.2,118
1,3.8,118
3,4.1,118
4,3.9,118
7,3.7,118


#### Size

In [13]:
# Size
sizes = list(df[df.columns[5]].unique())
sizes.sort(key=lambda x: int(re.findall('\d+', x)[0]))
replacement_index = {}
for i in range(len(sizes)):
    replacement_index[sizes[i]] = i
x['Employee'] = df[df.columns[6]].replace(replacement_index)
print(json.dumps(replacement_index, indent=4))
x.head()

{
    "1 to 50 employees": 0,
    "51 to 200 employees": 1,
    "201 to 500 employees": 2,
    "501 to 1000 employees": 3,
    "1001 to 5000 employees": 4,
    "5001 to 10000 employees": 5,
    "10000+ employees": 6
}


,Rating,Location,Employee
0,3.2,118,1961.0
1,3.8,118,1893.0
3,4.1,118,2002.0
4,3.9,118,2009.0
7,3.7,118,1914.0


#### Age

In [14]:
# Age
founded_dates = df[df.columns[6]].astype(int).to_numpy()
ages = datetime.date.today().year - founded_dates
x['Age'] = ages
x.head()

,Rating,Location,Employee,Age
0,3.2,118,1961.0,61
1,3.8,118,1893.0,129
3,4.1,118,2002.0,20
4,3.9,118,2009.0,13
7,3.7,118,1914.0,108


#### Ownership Type

In [15]:
# Ownership Type
ownership_types = np.sort(df[df.columns[7]].unique())
replacement_index = {}
for i in range(len(ownership_types)):
    replacement_index[ownership_types[i]] = i
x['Ownership Type'] = df[df.columns[7]].replace(replacement_index)
print(json.dumps(replacement_index, indent=4))
x.head()

{
    "College / University": 0,
    "Company - Private": 1,
    "Company - Public": 2,
    "Contract": 3,
    "Government": 4,
    "Hospital": 5,
    "Nonprofit Organization": 6,
    "Other Organization": 7,
    "Private Practice / Firm": 8,
    "School / School District": 9,
    "Subsidiary or Business Segment": 10
}


,Rating,Location,Employee,Age,Ownership Type
0,3.2,118,1961.0,61,6
1,3.8,118,1893.0,129,6
3,4.1,118,2002.0,20,10
4,3.9,118,2009.0,13,1
7,3.7,118,1914.0,108,1


#### Industry

In [16]:
# Industry
industries = np.sort(df[df.columns[8]].unique())
replacement_index = {}
for i in range(len(industries)):
    replacement_index[industries[i]] = i
x['Industry'] = df[df.columns[8]].replace(replacement_index)
print(json.dumps(replacement_index, indent=4))
x.head()

{
    "Accounting": 0,
    "Advertising & Marketing": 1,
    "Aerospace & Defense": 2,
    "Architectural & Engineering Services": 3,
    "Automotive Parts & Accessories Stores": 4,
    "Banks & Credit Unions": 5,
    "Biotech & Pharmaceuticals": 6,
    "Brokerage Services": 7,
    "Building & Personnel Services": 8,
    "Cable, Internet & Telephone Providers": 9,
    "Chemical Manufacturing": 10,
    "Colleges & Universities": 11,
    "Commercial Equipment Repair & Maintenance": 12,
    "Computer Hardware & Software": 13,
    "Construction": 14,
    "Consulting": 15,
    "Consumer Products Manufacturing": 16,
    "Convenience Stores & Truck Stops": 17,
    "Department, Clothing, & Shoe Stores": 18,
    "Drug & Health Stores": 19,
    "Education Training Services": 20,
    "Electrical & Electronic Manufacturing": 21,
    "Energy": 22,
    "Enterprise Software & Network Solutions": 23,
    "Federal Agencies": 24,
    "Financial Analytics & Research": 25,
    "Financial Transaction Proce

,Rating,Location,Employee,Age,Ownership Type,Industry
0,3.2,118,1961.0,61,6,59
1,3.8,118,1893.0,129,6,33
3,4.1,118,2002.0,20,10,36
4,3.9,118,2009.0,13,1,61
7,3.7,118,1914.0,108,1,39


#### Sector

In [17]:
# Sector
sectors = np.sort(df[df.columns[9]].unique())
replacement_index = {}
for i in range(len(sectors)):
    replacement_index[sectors[i]] = i
x['Sector'] = df[df.columns[9]].replace(replacement_index)
print(json.dumps(replacement_index, indent=4))
x.head()

{
    "Accounting & Legal": 0,
    "Aerospace & Defense": 1,
    "Arts, Entertainment & Recreation": 2,
    "Biotech & Pharmaceuticals": 3,
    "Business Services": 4,
    "Construction, Repair & Maintenance": 5,
    "Education": 6,
    "Finance": 7,
    "Government": 8,
    "Health Care": 9,
    "Information Technology": 10,
    "Insurance": 11,
    "Manufacturing": 12,
    "Media": 13,
    "Non-Profit": 14,
    "Oil, Gas, Energy & Utilities": 15,
    "Real Estate": 16,
    "Restaurants, Bars & Food Services": 17,
    "Retail": 18,
    "Telecommunications": 19,
    "Transportation & Logistics": 20
}


,Rating,Location,Employee,Age,Ownership Type,Industry,Sector
0,3.2,118,1961.0,61,6,59,14
1,3.8,118,1893.0,129,6,33,9
3,4.1,118,2002.0,20,10,36,10
4,3.9,118,2009.0,13,1,61,2
7,3.7,118,1914.0,108,1,39,11


#### Revenue

In [18]:
# Revenue
revenues = list(df[df.columns[-1]].unique())
revenues.sort(key=lambda x: (0 if 'million' in x else 1, int(re.findall('\d+', x)[0])))
replacement_index = {}
for i in range(len(revenues)):
    replacement_index[revenues[i]] = i
x['Revenue'] = df[df.columns[-1]].replace(replacement_index)
print(json.dumps(replacement_index, indent=4))
x.head()

{
    "$1 to $5 million (USD)": 0,
    "Less than $1 million (USD)": 1,
    "$5 to $10 million (USD)": 2,
    "$10 to $25 million (USD)": 3,
    "$25 to $50 million (USD)": 4,
    "$50 to $100 million (USD)": 5,
    "$100 to $500 million (USD)": 6,
    "$500 million to $1 billion (USD)": 7,
    "$1 to $2 billion (USD)": 8,
    "$2 to $5 billion (USD)": 9,
    "$5 to $10 billion (USD)": 10,
    "$10+ billion (USD)": 11
}


,Rating,Location,Employee,Age,Ownership Type,Industry,Sector,Revenue
0,3.2,118,1961.0,61,6,59,14,6
1,3.8,118,1893.0,129,6,33,9,9
3,4.1,118,2002.0,20,10,36,10,5
4,3.9,118,2009.0,13,1,61,2,6
7,3.7,118,1914.0,108,1,39,11,6


In [19]:
df.head()

,Job Title,Salary Estimate,Rating,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),3.2,"New York, NY","New York, NY",201 to 500 employees,1961.0,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD)
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),3.8,"New York, NY","New York, NY",10000+ employees,1893.0,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD)
3,Data Analyst,$37K-$66K (Glassdoor est.),4.1,"New York, NY","McLean, VA",201 to 500 employees,2002.0,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD)
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),3.9,"New York, NY","New York, NY",501 to 1000 employees,2009.0,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD)
7,Data Science Analyst,$37K-$66K (Glassdoor est.),3.7,"New York, NY","New York, NY",201 to 500 employees,1914.0,Company - Private,Insurance Carriers,Insurance,$100 to $500 million (USD)


### Train Data

In [20]:
y0 = y[y.columns[0]]
y0

0       37
1       37
2       37
3       37
4       37
        ..
1200    78
1201    78
1202    78
1203    78
1204    78
Name: Salary Lower, Length: 1205, dtype: int64

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y0, test_size=0.2)

## Prediction

In [22]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
predictions = decision_tree.predict(x_test)
score = accuracy_score(y_test, predictions)
score

0.25311203319502074

In [23]:
tree.export_graphviz(
    decision_tree,
    out_file='outputs/job-prediction.dot',
    feature_names=x.columns,
    class_names=sorted(y0.astype(str).unique()),
    label='all',
    rounded=True,
    filled=True,
)